In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt; plt.style.use('ggplot')

In [3]:
pd.set_option('display.max_columns', None)
data = pd.read_csv('/Users/willfitzhugh/Desktop/Coding/Supply-Chain-Analysis/Data/Interim/CleanedData.csv',
                   parse_dates=['OrderDate','ShipDate'])
data.head()

,PaymentType,RealShippingDays,ScheduledShippingDays,OrderProfit,OrderSales,Delivery Status,Late_delivery_risk,ProductCategory,Customer City,Customer Country,Customer Id,Customer Segment,Customer State,ProductDepartment,Latitude,Longitude,Market,Order City,Order Country,OrderDate,Order Id,Order Item Discount,Order Item Discount Rate,Order Item Profit Ratio,Order Item Quantity,Order Region,Order State,Order Status,Product Name,Product Price,ShipDate,Shipping Mode,CustomerName
0,DEBIT,3,4,91.250000,314.640015,Advance shipping,0,Sporting Goods,Caguas,Puerto Rico,20755,Consumer,PR,Fitness,18.251453,-66.037056,Pacific Asia,Bekasi,Indonesia,2018-01-31 22:56:00,77202,13.110000,0.04,0.29,1,Southeast Asia,Java Occidental,COMPLETE,Smart watch,327.75,2018-02-03 22:56:00,Standard Class,Cally Holloway
1,TRANSFER,5,4,-249.089996,311.359985,Late delivery,1,Sporting Goods,Caguas,Puerto Rico,19492,Consumer,PR,Fitness,18.279451,-66.037064,Pacific Asia,Bikaner,India,2018-01-13 12:27:00,75939,16.389999,0.05,-0.80,1,South Asia,Rajastn,PENDING,Smart watch,327.75,2018-01-18 12:27:00,Standard Class,Irene Luna
2,CASH,4,4,-247.779999,309.720001,Shipping on time,0,Sporting Goods,San Jose,EE. UU.,19491,Consumer,CA,Fitness,37.292233,-121.881279,Pacific Asia,Bikaner,India,2018-01-13 12:06:00,75938,18.030001,0.06,-0.80,1,South Asia,Rajastn,CLOSED,Smart watch,327.75,2018-01-17 12:06:00,Standard Class,Gillian Maldonado
3,DEBIT,3,4,22.860001,304.809998,Advance shipping,0,Sporting Goods,Los Angeles,EE. UU.,19490,Home Office,CA,Fitness,34.125946,-118.291016,Pacific Asia,Townsville,Australia,2018-01-13 11:45:00,75937,22.940001,0.07,0.08,1,Oceania,Queensland,COMPLETE,Smart watch,327.75,2018-01-16 11:45:00,Standard Class,Tana Tate
4,PAYMENT,2,4,134.210007,298.250000,Advance shipping,0,Sporting Goods,Caguas,Puerto Rico,19489,Corporate,PR,Fitness,18.253769,-66.037048,Pacific Asia,Townsville,Australia,2018-01-13 11:24:00,75936,29.500000,0.09,0.45,1,Oceania,Queensland,PENDING_PAYMENT,Smart watch,327.75,2018-01-15 11:24:00,Standard Class,Orli Hendricks


In [ ]:
data['late'] = data.RealShippingDays > data.ScheduledShippingDays
((data['Late_delivery_risk'] > .5) != (data.late)).sum()

In [10]:
data['ShipDate'] - data['OrderDate']

0        3 days
1        5 days
2        4 days
3        3 days
4        2 days
          ...  
180514   4 days
180515   3 days
180516   5 days
180517   3 days
180518   4 days
Length: 180519, dtype: timedelta64[ns]